In [2]:
import ee
import geemap

In [9]:
Map = geemap.Map(center=[-11.77,-45.76], zoom=8)
Map

Map(center=[-11.77, -45.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [40]:
def clusterization(image):

          
    # Load a pre-computed Landsat composite for input.
    input = image.clip(geometry).select("EVI");
  
    #Make the training dataset.
    training = input.sample(**{
    'region': geometry,
    'scale': 250,
    'numPixels': 5000
    });

    #Instantiate the clusterer and train it.
    clusterer = ee.Clusterer.wekaKMeans(5).train(training)
    #Cluster the input using the trained clusterer.
    result = input.cluster(clusterer)
 
    #Combine the mean and standard deviation reducers.
    reducers = ee.Reducer.mean()

    #Inicialização Lista de EVI dos clustes
    lista = ee.List([])

    #Iteração sobre os 5 Clusters para calcular a estatística de cada região
    for i in range(5):

        #Cálculo da geometria
        geom = result.select("cluster").eq(i).selfMask().reduceToVectors(**{
        'scale':250,
        });



        #Reducer para cálculo do EVI médio de cada cluster
        stats = image.reduceRegion(**{
        'reducer': reducers,
        'geometry':geom.geometry(),
        'scale':250,
        'bestEffort':True,
        'maxPixels':1000000000,
        });

        #Armarzena o EVI médio de cada clúster em uma lista
        lista = lista.add(stats.get("EVI"));


    #Cópia da Lista para ordenar os 2 maiores EVIs
    lista2 = lista;


    #Ordenar a lista para pegar os 2 maiores valores de EVI médio nos clusters
    #Agora nossos clustests de interesse estão nas posições 3 e 4 da lista 2
    lista2 = lista2.sort()

    #Pegar as chaves dos clusteres de interesse na lista original
    chaves1 = lista.indexOf(lista2.get(3))
    chaves2 = lista.indexOf(lista2.get(4))


    #Cálculo da Geometria final

    geom1 = result.select("cluster").eq(chaves1).selfMask().reduceToVectors(**{
    'scale':250,
    });

    geom2 = result.select("cluster").eq(chaves2).selfMask().reduceToVectors(**{
    'scale':250,
    });

    #Geom é a feature collection de nossos clusters de interesse
    geom = geom1.merge(geom2);



    stats = image.reduceRegion(**{
    'reducer': reducers,
    'geometry':geom.geometry(),
    'scale':250,
    'bestEffort':True,
    'maxPixels':1000000000,
    });


    return image.set('EVI_Cluster', stats.get("EVI"))
    

In [11]:
Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map

Map(bottom=35226.0, center=[-11.77, -45.76], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

In [22]:
reducers = ee.Reducer.mean()


# Reducer para cálculo do EVI médio de cada cluster
stats = lst.reduceRegion(**{
  'reducer': reducers,
  'geometry':region,
  'scale':250,
});

In [24]:
print(stats.getInfo())

{'NDVI': 8108.787373042909}


In [62]:
    #Filtrar a territorialidade 
    mcollection = ee.ImageCollection('MODIS/006/MOD13Q1').filterBounds(geometry).filterDate('2011-01-01', '2021-04-30').filter(ee.Filter.calendarRange(1, 4,'month')).map(clusterization)
    #4 principais meses de safra
    #filter start and end date
    
    

In [43]:
import pandas as pd

In [63]:
a = mcollection.aggregate_array('EVI_Cluster')

In [64]:
x = a.getInfo()

EEException: Computation timed out.

In [ ]:
df = pd.DataFrame({'lin':x})

In [ ]:
df